
### Step 1: Connect to ADLS

Since we are using Azure Credentials Passthrough connection method, we just need to use the custom access token and mount the folder that has all our TestStand files. 

In [ ]:
# Install fsspec if needed
# %pip install fsspec

# Declare script variables
database_name = "dbo"

bronze_source = "REDACTED"
bronze_target = bronze_source + database_name
bronze_mnt = "/mnt/bronze/" + database_name

silver_source = "REDACTED"
silver_target = silver_source + database_name
silver_mnt = "/mnt/silver/" + database_name

gold_source = "REDACTED"
gold_target = gold_source + database_name
gold_mnt = "/mnt/gold/" + database_name

# Azure Credentials Passthrough configuration.
configs = {
  "fs.azure.account.auth.type": "CustomAccessToken",
  "fs.azure.account.custom.token.provider.class": spark.conf.get("spark.databricks.passthrough.adls.gen2.tokenProviderClassName")
}



### Step 2: Mount bronze container


In [ ]:
# Add bronze_mnt if not already mounted.
if any(mount.mountPoint == bronze_mnt for mount in dbutils.fs.mounts()):
    pass
else:
    dbutils.fs.mount(
        source=bronze_source, mount_point=bronze_mnt, extra_configs=configs
    )

# List files in directory
dbutils.fs.ls(bronze_target)

[FileInfo(path='abfss://bronze@arcdatalakeg2.dfs.core.windows.net/dbo/PROP_ANALOGWAVEFORM.parquet', name='PROP_ANALOGWAVEFORM.parquet', size=343, modificationTime=1708270317000),
 FileInfo(path='abfss://bronze@arcdatalakeg2.dfs.core.windows.net/dbo/PROP_BINARY.parquet', name='PROP_BINARY.parquet', size=1044738, modificationTime=1708270299000),
 FileInfo(path='abfss://bronze@arcdatalakeg2.dfs.core.windows.net/dbo/PROP_DIGITALWAVEFORM.parquet', name='PROP_DIGITALWAVEFORM.parquet', size=311, modificationTime=1708270284000),
 FileInfo(path='abfss://bronze@arcdatalakeg2.dfs.core.windows.net/dbo/PROP_NUMERICLIMIT.parquet', name='PROP_NUMERICLIMIT.parquet', size=1289983, modificationTime=1708270268000),
 FileInfo(path='abfss://bronze@arcdatalakeg2.dfs.core.windows.net/dbo/PROP_RESULT.parquet', name='PROP_RESULT.parquet', size=23944627, modificationTime=1708270257000),
 FileInfo(path='abfss://bronze@arcdatalakeg2.dfs.core.windows.net/dbo/STEP_RESULT.parquet', name='STEP_RESULT.parquet', size=1


### Step 3: Mount silver container


In [ ]:
# Add silver_mnt if not already mounted.
if any(mount.mountPoint == silver_mnt for mount in dbutils.fs.mounts()):
    pass
else:
    dbutils.fs.mount(
        source=silver_source, mount_point=silver_mnt, extra_configs=configs
    )

# List files in directory
dbutils.fs.ls(silver_target)

[]


### Step 4: Mount gold container


In [ ]:
# Add gold_mnt if not already mounted.
if any(mount.mountPoint == gold_mnt for mount in dbutils.fs.mounts()):
    pass
else:
    dbutils.fs.mount(
      source=gold_source, mount_point=gold_mnt, extra_configs=configs
    )

# List files in directory
dbutils.fs.ls(gold_target)

---------------------------------------------------------------------------
ExecutionError                            Traceback (most recent call last)
File <command-2461738334509896>, line 10
      5     dbutils.fs.mount(
      6         source=bronze_source, mount_point=bronze_mnt, extra_configs=configs
      7     )
      9 # List files in directory
---> 10 dbutils.fs.ls(bronze_target)

File /databricks/python_shell/dbruntime/dbutils.py:378, in DBUtils.FSHandler.prettify_exception_message.<locals>.f_with_exception_handling(*args, **kwargs)
    376 exc.__context__ = None
    377 exc.__cause__ = None
--> 378 raise exc

ExecutionError: An error occurred while calling o406.ls.
: Failure to initialize configuration for storage account arcdatalakeg2.dfs.core.windows.net: Invalid configuration value detected for fs.azure.account.keyInvalid configuration value detected for fs.azure.account.key
	at shaded.databricks.azurebfs.org.apache.hadoop.fs.azurebfs.services.SimpleKeyProvider.getStorage


### Step 5: Setup Database and Extract Data to Tables in Databricks

We instantiate a DBFS database and USE it. Then we iterate over the TSDB directory at that level and create a table for each CSV file in that mount point. 


In [ ]:
# Create and use DATABASE [drop IF ALREADY EXISTS]
spark.sql(f"drop database IF EXISTS {database_name} cascade ")
spark.sql(f"create database {database_name}")
spark.sql(f"use {database_name}")

# Create all tables into DBFS database
tables_list = dbutils.fs.ls(bronze_target)

for table in tables_list:
    table_name = table.name.strip(".parquet")
    spark.sql(f"DROP TABLE IF EXISTS `{table_name}`")
    spark.sql(f"""
              CREATE TABLE `{table_name}` 
              USING parquet 
              OPTIONS(path '{table.path}', header 'true', inferschema 'true')
              """)
    
# Display table
if debug_mode:
    df = spark.sql('select * from uut_result')
    display(df)

---------------------------------------------------------------------------
ExecutionError                            Traceback (most recent call last)
File <command-2461738334509896>, line 10
      5     dbutils.fs.mount(
      6         source=bronze_source, mount_point=bronze_mnt, extra_configs=configs
      7     )
      9 # List files in directory
---> 10 dbutils.fs.ls(bronze_target)

File /databricks/python_shell/dbruntime/dbutils.py:378, in DBUtils.FSHandler.prettify_exception_message.<locals>.f_with_exception_handling(*args, **kwargs)
    376 exc.__context__ = None
    377 exc.__cause__ = None
--> 378 raise exc

ExecutionError: An error occurred while calling o406.ls.
: Failure to initialize configuration for storage account arcdatalakeg2.dfs.core.windows.net: Invalid configuration value detected for fs.azure.account.keyInvalid configuration value detected for fs.azure.account.key
	at shaded.databricks.azurebfs.org.apache.hadoop.fs.azurebfs.services.SimpleKeyProvider.getStorage